# Final Exam

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 155 points.
* You have 180 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Final Exam` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `final_exam_data.xlsx`

This file has sheets for...
* `portfolio` (weekly) - Part 2
* `forecasting` (monthly) - Part 3
* `fx_carry`(daily) - Part 4

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 40     |
| 2       | 25     |
| 3       | 50     |
| 4       | 40     |

### Each numbered question is worth 5 points unless otherwise specified.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\betamkt}{\beta^{i,\text{MKT}}}$$
$$\newcommand{\betahml}{\beta^{i,\text{HML}}}$$
$$\newcommand{\betaumd}{\beta^{i,\text{UMD}}}$$
$$\newcommand{\Eri}{E\left[\tilde{r}^{i}\right]}$$
$$\newcommand{\Emkt}{E\left[\tilde{r}^{\text{MKT}}\right]}$$
$$\newcommand{\Ehml}{E\left[\tilde{r}^{\text{HML}}\right]}$$
$$\newcommand{\Eumd}{E\left[\tilde{r}^{\text{UMD}}\right]}$$

$$\newcommand{\frn}{\text{MXN}}$$
$$\newcommand{\frnrate}{\text{MXSTR}}$$
$$\newcommand{\FXspot}{S}$$
$$\newcommand{\fxspot}{\texttt{s}}$$
$$\newcommand{\rflogusd}{\texttt{r}^{\text{USD}}}$$
$$\newcommand{\rflogfrn}{\texttt{r}^{\frn}}$$

$$\newcommand{\wintt}{t,t+1}$$

$$\newcommand{\targ}{\text{USO}}$$

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.

Consider a Linear Factor Pricing Model (LFPM).

Which metric do we examine to understand its fit, (or errors)...
* given the estimated **time-series (TS)** test?
* given the estimated **cross-sectional (CS)** test?

### 2.

Consider the Arbitrage Pricing Theory (APT). Is it fair to say that it is more likely to work for sets of assets with low cross-correlation? Why or why not?

### 3.

In constructing momentum portfolios, we discussed selecting the top and bottom 10% of stocks, ranked by past returns. How do you think the strategy would be impacted if we were more extreme in the selection, and went long-short just the top / bottom 1% of total stocks?

### 4.

Over longer horizons, do investments have higher Sharpe ratios? How is this issue relevant to long-term asset allocators such as Barnstable?

### 5.

Before it crashed, how did LTCM's performance compare to the S&P (SPY)? Was it an attractive investment? Be specific.

### 6.

Suppose investors are **not** mean-variance investors. If we find an investment with a Sharpe ratio higher than the "market", would this would be inconsistent with the CAPM?

### 7.

What causes us concern about the performance of classic mean-variance optimization out-of-sample?

What is one of the potential solutions we discussed?

### 8.

True or False: Uncovered Interest Parity implies Covered Interest Parity, but not vice-versa.

Explain.

***

In [288]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [289]:
portfolio_df = pd.read_excel("practice final_exam_data.xlsx",sheet_name="portfolio").set_index("date")
portfolio_df.head()

,SPY,BTC,USO,TLT,IEF,IYR,GLD
date,,,,,,,
2016-01-15,-0.021430,-0.143915,-0.103061,0.019391,0.007840,-0.030714,-0.015140
2016-01-22,0.014429,-0.021378,0.054608,-0.003277,-0.001111,0.010138,0.008839
2016-01-29,0.016801,-0.010105,0.040992,0.015638,0.011495,0.010527,0.018564
2016-02-05,-0.029789,0.025800,-0.076684,0.013072,0.007335,-0.026806,0.050218
2016-02-12,-0.007023,-0.008470,-0.065095,0.021597,0.007563,-0.041958,0.053775


In [290]:
forecasting_df = pd.read_excel("practice final_exam_data.xlsx",sheet_name="forecasting").set_index("date")
forecasting_df.head()

,USO,Tnote rate,Tnote rate change
date,,,
2009-05-31,0.271394,3.465,0.341
2009-06-30,0.042033,3.523,0.058
2009-07-31,-0.029528,3.501,-0.022
2009-08-31,-0.020647,3.401,-0.100
2009-09-30,0.003883,3.307,-0.094


In [291]:
fx_df = pd.read_excel("practice final_exam_data.xlsx",sheet_name="fx_carry").set_index("date")
fx_df.head()

,MXN,SOFR,MXSTR
date,,,
2018-04-03,0.054959,0.000073,0.000298
2018-04-04,0.054852,0.000069,0.000298
2018-04-05,0.055166,0.000069,0.000298
2018-04-06,0.054558,0.000069,0.000298
2018-04-09,0.054871,0.000069,0.000298


## Helper Functions

### Performance Summary

In [292]:
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

### Time-series Regression

In [293]:
def time_series_regression(portfolio, factors, multiple_factors = False, resid = False, annualization = 1):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const']*annualization
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
        else:
            ff_report.loc[portf, factors.name + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] =  annualization* portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

### Tangency Weights

In [294]:
def tangency_weights(returns, cov_mat = 1):
    
    if cov_mat ==1:
        cov_inv = np.linalg.inv((returns.cov()*12))
    else:
        cov = returns.cov()
        covmat_diag = np.diag(np.diag((cov)))
        covmat = cov_mat * cov + (1-cov_mat) * covmat_diag
        cov_inv = np.linalg.inv((covmat*12))  
        
    ones = np.ones(returns.columns[:].shape) 
    mu = returns.mean()*12
    scaling = 1/(np.transpose(ones) @ cov_inv @ mu)
    tangent_return = scaling*(cov_inv @ mu) 
    tangency_wts = pd.DataFrame(index = returns.columns[:], data = tangent_return, columns = ['Tangent Weights'] )
        
    return tangency_wts

### Global-Minimum Variance Weights

In [295]:
def gmv_weights(tot_returns):
    
    ones = np.ones(tot_returns.columns[:].shape)
    cov = tot_returns.cov()*12
    cov_inv = np.linalg.inv(cov)
    scaling = 1/(np.transpose(ones) @ cov_inv @ ones)
    gmv_tot = scaling * cov_inv @ ones
    gmv_wts = pd.DataFrame(index = tot_returns.columns[:], data = gmv_tot, columns = ['GMV Weights'] )

    
    return gmv_wts


### Mean-Variance Portfolio

In [296]:
def mv_portfolio_excess_returns(target_ret, ex_ret):
    
    mu_tilde = ex_ret.copy().mean()
    
    Sigma_adj = ex_ret.copy().cov()
    
    Sigma_inv = np.linalg.inv(Sigma_adj)
    
    N = Sigma_adj.shape[0]
    
    delta_tilde = ((np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde)) * target_ret
    
    omega_star = delta_tilde * tan_wts
    
    return omega_star
    

In [297]:
def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, cov_mat = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, cov_mat = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    
    MV = pd.DataFrame(index = tot_returns.columns[:], data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, cov_mat = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values


    return MV


### OOS forecast strategy

In [298]:
def OOS_strat(df, factors, start):
    y = df
    X = sm.add_constant(factors)

    forecast_err, null_err,oos_predictions,null_predictions = [], [],[],[]

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            oos_predictions.append(reg_predict.T)
            null_predictions.append(pd.DataFrame([[reg_predict.index[0]]], columns = ['date'], index = [null_forecast]))
            forecast_err.append(reg_predict.values - actual)
            null_err.append(null_forecast.values - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    predictions_df = pd.concat(oos_predictions).T.drop_duplicates()
    null_predictions_df = pd.concat(null_predictions).T
    
    return ((1 - RSS/TSS),reg,predictions_df,null_predictions_df)

### OOS R-squared

In [299]:
def OOS_r2(df, factors, start):
    y = df['SPY']
    X = sm.add_constant(factors)

    forecast_err, null_err = [], []

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return ((1 - RSS/TSS),reg)

# 2. Optimization

Use the data found in the `portfolio` tab. it is weekly data.

### 1.

Assume the provided data is in terms of **excess** returns.

Report the weights of the tangency portfolio.

Report the weights of the MV portfolio which achieves a mean weekly return of `0.0025`.

In [300]:
target_rt = 0.0025

tan_wts = tangency_weights(portfolio_df)
tan_wts

,Tangent Weights
SPY,0.857595
BTC,0.141710
USO,-0.043098
TLT,-0.040488
IEF,0.167449
IYR,-0.440830
GLD,0.357662


In [301]:
mv_er = mv_portfolio_excess_returns(target_rt, portfolio_df)
mv_er

,Tangent Weights
SPY,0.479969
BTC,0.079311
USO,-0.024120
TLT,-0.022660
IEF,0.093716
IYR,-0.246719
GLD,0.200172


### 2.

Assume the provided data is in terms of **total** returns.

Report the weights of the GMV portfolio

Report the weights of the MV portfolio which achieves a mean weekly return of `0.0025`.

In [302]:
gmv_wts = gmv_weights(portfolio_df)
gmv_wts

,GMV Weights
SPY,0.092076
BTC,-0.001009
USO,0.002750
TLT,-0.476953
IEF,1.427028
IYR,-0.041140
GLD,-0.002752


### 3.

Conceptually, what is the difference between the portfolios in part 1 and part 2?

Mathematically, what is the difference in their optimizations?

### 4.

#### (10pts)

Consider the following:
* drop `BTC` from the sample
* target a weekly mean return of `.0025`.
* assume once again that the provided data is **excess** returns.

Using data only through 2021, 
* calculate the tangency weights
* compute the performance of this tangency portfolio in the out-of-sample (OOS) period of 2022-2023.

Report the
* mean
* vol
* Sharpe

Compare these three metrics with the equally-weighted portfolio for 2022-2023.

In [303]:
excess_ret_wo_BTC = portfolio_df.drop(columns = {'BTC'})

excess_ret_wo_BTC_IS = excess_ret_wo_BTC.loc[:'2021']
excess_ret_wo_BTC_OOS = excess_ret_wo_BTC.loc['2022':'2023']

In [304]:
wts = pd.DataFrame(index = excess_ret_wo_BTC_IS.columns, columns = ['tangency','equal weights'])

wts.loc[:,'tangency'] = tangency_weights(excess_ret_wo_BTC_IS, cov_mat = 1)
wts.loc[:,'equal weights'] = 1/len(excess_ret_wo_BTC_IS.columns)
wts *= target_rt / (excess_ret_wo_BTC_IS.mean()@wts)

wts

,tangency,equal weights
SPY,0.70823,0.242376
USO,-0.03479,0.242376
TLT,-0.279598,0.242376
IEF,1.71295,0.242376
IYR,-0.252887,0.242376
GLD,0.023858,0.242376


In [305]:
a = performance_summary(excess_ret_wo_BTC_OOS @ wts[['tangency']], 52)
b = performance_summary(excess_ret_wo_BTC_OOS @ wts[['equal weights']], 52)

In [306]:
a.iloc[:,:3]

,Mean,Volatility,Sharpe Ratio
tangency,-0.07043,0.163527,-0.430691


In [307]:
b.iloc[:,:3]

,Mean,Volatility,Sharpe Ratio
equal weights,-0.036569,0.183591,-0.199185


# 3.

Forecast (total) returns on gold as tracked by the ETF ticker, $\targ$. This ETF holds crude oil.

As signals, use two interest rate signals, as seen in Treasury-notes. (No need to consider anything specific about Treasury notes, just read these as macroeconomic signals.)
* Tnote rate
* month-over-month change in the Tnote rate

Find the all data needed for this problem in the sheet `forecasting`.

### 1.

Estimate a forecasting regression of $\targ$ on the two (lagged) signals.

$$r_{t+1}^\targ = \alpha + \beta^{x}x_t + \beta^z z_t + \epsilon_{t+1}$$

where
* $x$ denotes the interest-rate signal.
* $z$ denotes the change in rate signal.

Report the r-squared, as well as the OLS estimates for the intercept and the two betas. (No need to annualize the stats.)

### 2.

Use your forecasted returns, $\hat{r}^{\targ}_{t+1}$ to build trading weights:

$$w_t = 0.50 + 50\;\hat{r}^{\targ}_{t+1}$$

(So the rule says to hold 50% in the ETF plus/minus 50x the forecast. Recall the forecast is a monthly percentage, so it is a small number.)

Calculate the return from implementing this strategy. Denote this as $r^x_t$.

Report the first and last 5 values.

### 3.

Calculate the following (annualized) performance metrics for both the passive investment, $r^\targ$, as well as the strategy implemented in the previous problem, $r^x$.

* mean
* volatility
* max drawdown

### 4.

#### (7pts)


Suppose we are assessing the returns to this active strategy, $r^x$, without knowing how it is generated. 

Use a regression (with an intercept) to report the optimal hedge ratio of passive $\targ$ to this active strategy. 

* Report the hedge ratio, being clear about whether you are going long or short $\targ$ in order to hedge.

* What is the mean return of the hedged active strategy?

### 5.

#### (8pts)

For the rest of the problem, consider the out-of-sample (OOS) performance of the strategy.

Forecast values of $\targ$ for January 2018 through Dec 2023. (So we are using the data up until January 2018 as “burn-in” data.)
* Loop through time, estimating the forecast only using data through time $t$.
* At each step, calculate the next OOS forecast, $\hat{r}^{\targ}_{t+1}$.

Report the first and last 5 values of your OOS forecast, $\hat{r}^{\targ}_{t+1}$.

### 6. 

#### (8pts)

Report the out-of-sample r-squared, relative to a baseline forecast which is simply the mean of $\targ$ up to the point the forecast is made.

Does the forecast seem effective?

### 7. 

Report the correlation between 
* OOS forecast
* realized value of $\targ$.

In light of this, how effective does the forecast seem?

### 8.

#### (7pts)

Convert your OOS forecast to a traded return strategy, using the same allocation rule as in part 2.

Report the following performance stats for the OOS forecast strategy.

* mean
* volatility
* max-drawdown

Compare these with the passive return, $r^\targ$ over the same OOS window.

***

# 4. 

We examine FX carry for trading the Mexican peso $\frn$.
* Find the FX and risk-free rate data for this problem on sheet `fx_carry`. As before, these are spot FX prices quoted as USD per $\frn$.
* SOFR is the risk-free rate on USD, and $\frnrate$ is the risk-free rate for $\frn$.
* As in Homework 8, the data is provided such that any row’s date, $t$, is reporting $S_t$ and $r^f_{t,t+1}$.
That is, both of these are known at time t.

### 1.
#### (3pts)

Transform the data to **log** FX prices and **log** interest rates, just as we did in Homework 8.

$$\begin{align}
\fxspot_t & \equiv \ln\left(\FXspot_t\right)\\[3pt]
\rflogusd_{\wintt} & \equiv \ln\left(1+r^{\text{USD}}_{\wintt}\right)\\[3pt]
\rflogfrn_{\wintt} & \equiv \ln\left(1+r^{\frn}_{\wintt}\right)\\
\end{align}$$


Display the mean of all three series.

### 2.

Calculate the excess log return to a USD investor of holding $\frn$. Report the following **annualized** stats...
* Mean
* Volatility
* Sharpe ratio.

Assume there are 252 reported days per year for pursposes of annnualization.

### 3. 

Over the sample, was it better to be long or short $\frn$ relative to USD?
* Did the interest spread help on average?
* Did the USD appreciate or depreciate relative to $\frn$ over the sample?

### 4.

#### (7pts)

Forecast the growth of the FX rate using the interest-rate differential:

$$\fxspot_{t+1} - \fxspot_t = \alpha + \beta\left(\rflogusd_{\wintt} - \rflogfrn_{\wintt}\right) + \epsilon_{t+1}$$

Report the following OLS stats, (no need to annualize or scale them.)
* $\alpha$
* $\beta$
* r-squared

### 5. 

If we assume the Uncovered Interest Parity to hold true, what would you expect to be true of the regression estimates?

### 6.

Based on the regression results, if we observe an increase in the interest rate on USD relative to $\frn$, should we expect the USD to get stronger (appreciate) or weaker (depreciate)?

### 7.

If the risk free rates in $\frn$ increase relative to risk-free rates in USD, do we expect the forward exchange rate to be higher than the spot exchange rate?

### 8.

Do you think the estimated forecast impact of rates on currency returns would be larger over an annual horizon instead of a daily horizon? Why?

***